In [1]:
# LIBRARIES

import pandas as pd
import os
import requests

In [2]:
# CONFIG
os.chdir('/Users/pportocarrero/OneDrive/Projects/Covid-19')

print('Current working directory: {0}'.format(os.getcwd()))

Current working directory: /Users/pportocarrero/Library/CloudStorage/OneDrive-Personal/Projects/Covid-19


In [4]:
# GET NEW DATA

req = requests.get('https://datos.ins.gob.pe/dataset/1816a9a4-5ef9-437a-ac46-248e3b6a7ef6/resource/424d9e7a-c2f7-44f4-9a99-115b0b932918/download/pmgenoma_04feb2022.csv')
url_content = req.content
csv_file = open('lineages.csv', 'wb')
csv_file.write(url_content)
csv_file.close()

SSLError: HTTPSConnectionPool(host='datos.ins.gob.pe', port=443): Max retries exceeded with url: /dataset/1816a9a4-5ef9-437a-ac46-248e3b6a7ef6/resource/424d9e7a-c2f7-44f4-9a99-115b0b932918/download/pmgenoma_04feb2022.csv (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)')))

In [ ]:
# DATA WRANGLING

data = pd.read_csv('lineages.csv', delimiter = '|')

data['FECHA_MUESTRA'] = data['FECHA_MUESTRA'].astype(float, errors = 'raise')

nan_values = data['FECHA_MUESTRA'].isna().sum()

data['FECHA_MUESTRA'] = data['FECHA_MUESTRA'].fillna('20211231')

data['FECHA_MUESTRA'] = data['FECHA_MUESTRA'].astype(float, errors = 'raise')

data['FECHA_MUESTRA'] = data['FECHA_MUESTRA'].map('{:.0f}'.format)

data['FECHA_MUESTRA'] = data['FECHA_MUESTRA'] * 1

# SOME MORE WORK ON THE DATA

data['YEAR'] = data['FECHA_MUESTRA'].str[2:4]

data['MONTH'] = data['FECHA_MUESTRA'].str[4:6]

data['DAY'] = data['FECHA_MUESTRA'].str[6:8]

data['DATE'] = data['DAY'] + '/' + data['MONTH'] + '/' + data['YEAR']

data['DATE'] = pd.to_datetime(data['DATE'])

data['DATE'] = pd.to_datetime(data['DATE']).dt.strftime('%d/%m/%y')

data["DATE"] = data["DATE"].astype("datetime64")

In [ ]:
# AGGREGATES

aggregate_lineages = data.groupby(['RESULTADO']).size()

data['time'] = pd.to_datetime(data['DATE'])
date = data['time'].dt.date
date = date.drop_duplicates()
date = date.astype("datetime64")
date = date.sort_values(ascending=True)

df_aggregate = pd.DataFrame({'casos': aggregate_lineages, 'fecha': pd.date_range('', periods = len(aggregate_lineages))})

df_aggregate.to_excel('linajes.xlsx')